In [1]:
import json
import requests
from datetime import datetime, timedelta 
from lxml import etree
import json
from base64 import b64decode
import pandas as pd

# 从ixigua收集信息

In [2]:
user_id_list = [
#     4492956276,
#     99849181082,
#     104246645773
    5757425042
]

In [3]:
def downloadfile(filepath, url):
    print('downloadfile %s' % (url))
    r=requests.get(url, stream=True)
    print("****Connected****")
    f=open(filepath,'wb');
    print("Donloading.....")
    for chunk in r.iter_content(chunk_size=255):
        if chunk: # filter out keep-alive new chunks
            f.write(chunk)
    print("Done")
    f.close()

def request_detail_page(path, item=None):
    cookies = {
        'wafid':'af2ade58-1768-4fea-983d-c2ad0cf69fc2',
        'wafid.sig': 'xXI_Dn80Pyi_4TsddkTjVMYvnAQ',
        'xiguavideopcwebid': '6798376759479977479',
        'xiguavideopcwebid.sig': 'reKWpJ4q1-zRPqWZ3roOINB3wGc'
    }


    headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'zh-CN,zh;q=0.9,en;q=0.8,ja;q=0.7,ko;q=0.6,zh-TW;q=0.5',
        'cache-control': 'max-age=0',
        'referer': 'https://www.ixigua.com/',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-user': '?1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36'
    }

    print('https://www.ixigua.com%s' % (path))
    r = requests.get('https://www.ixigua.com%s' % (path), headers=headers, cookies=cookies)
    html = etree.HTML(r.text)
    str_list = html.xpath('.//script/text()')
    for one in str_list:
        if 'video_1' in one:
            content = one.replace('window._SSR_HYDRATED_DATA=', '')

            o = json.loads(content)
            video_id = o['Projection']['video']['videoResource']['normal']['video_id']
            video_url = b64decode(o['Projection']['video']['videoResource']['normal']['video_list']['video_3']['main_url']).decode()

            downloadfile('%s.mp4' % (video_id), video_url)
            
            if item is None:
                item = o['Projection']['video']
                
            item['video_id'] = video_id
            return item

In [4]:
def main():
    headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'zh-CN,zh;q=0.9,en;q=0.8,ja;q=0.7,ko;q=0.6,zh-TW;q=0.5',
        'cache-control': 'max-age=0',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'none',
        'sec-fetch-user': '?1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 13_2_3 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0.3 Mobile/15E148 Safari/604.1'
    }

    results = []
#     ct = datetime.now() - timedelta(days = 1) 
    ct = datetime.now()
    for user_id in user_id_list:
        min_behot_timestamp = 0
        for i in range(0, 3):
            url = 'https://m.ixigua.com/video/app/user/home/?to_user_id=%d&format=json&max_behot_time=%d' % (user_id, min_behot_timestamp)

            r = requests.get(url, headers=headers)
            o = json.loads(r.text)

            if not 'data' in list(o.keys()) or o['data'] is None or len(o['data']) == 0:
                continue

            min_behot_timestamp = o['data'][0]['behot_time']
            for item in o['data']:
                timestamp = item['behot_time']
                behot_time = datetime.fromtimestamp(timestamp) 

                if min_behot_timestamp > timestamp:
                    min_behot_timestamp = timestamp

                if behot_time.month == ct.month and behot_time.day == ct.day and behot_time.year == ct.year:
                    
                    try:
                        vitem = request_detail_page('/i%s' % (item['item_id']), item=item)
                        results.append(vitem)
                    except BaseException as e:
                        print(e)

    return results


In [5]:
# vitem = request_detail_page('/i%s' % ('6799538518235283470'))
# vitem

data = main()

Done
https://www.ixigua.com/i6799898162195595790
downloadfile http://v3-tt.ixigua.com/60876cf226f46cd2b458fe21419ec34a/5e5e4a3e/video/tos/cn/tos-cn-ve-31/dab4eab01a8a407abfa30a145548684d/?a=1768&br=0&bt=1037&cr=0&cs=0&dr=0&ds=3&er=&l=2020030319142901001401810909074F65&lr=&qs=0&rc=anc8OjxrNG80czMzaWkzM0ApNmg2ZTo3ZmQ0Nzo5aWk1aGcybTAvbl5kYmNfLS1eLi9zc18vXmEyMF5fMDMyL2E2Li46Yw%3D%3D&vl=&vr=
****Connected****
Donloading.....
Done
https://www.ixigua.com/i6799897938467226125
downloadfile http://v3-tt.ixigua.com/500ad6e6a916562a96f964ac3570e7d3/5e5e4a38/video/tos/cn/tos-cn-ve-31/1dada39dac004ce881ec9ff562f88007/?a=1768&br=0&bt=996&cr=0&cs=0&dr=0&ds=3&er=&l=202003031914300100120631652009C190&lr=&qs=0&rc=M2Z5cWVlajw0czMzOGkzM0ApOGVoNzlpN2UzN2k7ZTY0OGdiNm4wZWsvYmNfLS1fLi9zc2E2YzE2NTFeXzMzLWMzYTQ6Yw%3D%3D&vl=&vr=
****Connected****
Donloading.....
Done
https://www.ixigua.com/i6799897792438338062
downloadfile http://v3-tt.ixigua.com/1c22f84365a6f8e3fe618aa5c77c9dd6/5e5e4a3e/video/tos/cn/tos-cn-ve-31

Done
https://www.ixigua.com/i6799880949787001357
downloadfile http://v3-tt.ixigua.com/85e400a703d6c6a2d593764c5bf39648/5e5e4aba/video/tos/cn/tos-cn-ve-31/799e28b955fb414ea56660fdb0bc649e/?a=1768&br=0&bt=928&cr=0&cs=0&dr=0&ds=3&er=&l=20200303191548010017087043000935F1&lr=&qs=0&rc=ajd5PHNnNWUzczMzaWkzM0ApZjtnOmQ1OzszNzY2aWc2NGdfMWtmYWRzY2NfLS1hLi9zczMwYF8zYmNhMzM2YmMtYmE6Yw%3D%3D&vl=&vr=
****Connected****
Donloading.....
Done
https://www.ixigua.com/i6799880828814885384
'video_3'
https://www.ixigua.com/i6799880788406960647
downloadfile http://v3-tt.ixigua.com/ceab6bd9eabb4bea64fb9301bb7995ff/5e5e4a9a/video/tos/cn/tos-cn-ve-31/a45ab90bdd2648d3840eabd92fc4cef0/?a=1768&br=0&bt=994&cr=0&cs=0&dr=0&ds=3&er=&l=20200303191549010017088105230B381F&lr=&qs=0&rc=ajV1Zzw6aTUzczMzNWkzM0ApaTdoOmc6ZWU6NzM3Zzc5aGdhMV8yNGM0Y2NfLS1fLi9zczJgMy81NTIzMV80XjYzYi06Yw%3D%3D&vl=&vr=
****Connected****
Donloading.....
Done
https://www.ixigua.com/i6799880526778860046
'video_3'
https://www.ixigua.com/i67998795252252021

In [6]:
tmp = []
for partial in data:
    tmp.append({
        'video_id': partial['video_id'],
        'abstract': partial['abstract'],
        'title': partial['title'],
        'media_name': partial['media_name']
    })

df = pd.DataFrame(tmp)
df

,video_id,abstract,title,media_name
0,v0201ffb0000bpf3hq3bo1i6f3d09mdg,原标题:科画 | 为什么狗鼻子是凉的？英媒：那是超灵敏的热探测器狗的鼻子为什么是凉的？英媒称...,科画 | 为什么狗鼻子是凉的？英媒：那是超灵敏的热探测器,新华社
1,v0201fb30000bpf3fvfeqk8dslufu60g,狗的鼻子为什么是凉的？英媒称，这个谜团似乎已经解开了：因为它们是超灵敏的热探测器。据英国《泰...,科画 | 为什么狗鼻子是凉的？英媒：那是超灵敏的热探测器,新华社
2,v0201fd10000bpf3efa0ifkiuoqlu8tg,人社部等三部门联合发布智能制造工程技术人员等16个新职业,人社部等三部门联合发布智能制造工程技术人员等16个新职业,新华社
3,v0201f0f0000bpf3e1mdm157h62ipka0,上海：规模以上工业企业复工率已近95%,上海：规模以上工业企业复工率已近95%,新华社
4,v0201f1e0000bpf3e1ld2r69d3hg381g,2019年北京市常住人口2153.,2019年北京市常住人口2153.6万人,新华社
...,...,...,...,...
80,v0201faf0000bpf003cqs4sh0e82avjg,近日，南京、武汉2座城市11所高校的50名大学生乐团成员，以线上“云合奏”的方式演绎了江苏民...,南京武汉2城11所高校大学生乐团“云上合奏”两地经典,新华社
81,v0201f020000bpevvg51mikdrridon0g,大卫·霍克尼素描展在伦敦国家肖像美术馆展出,大卫·霍克尼素描展在伦敦国家肖像美术馆展出,新华社
82,v0201fcc0000bpevuag2saj9vtbj6730,第42届温哥华国际葡萄酒节圆满落幕,第42届温哥华国际葡萄酒节圆满落幕,新华社
83,v0201f0f0000bpevt9suatlbvbvk4b20,近期，河北省广宗县第三中学教师郭丽谭创作了“致敬钟南山”“春暖花开”“愿平安归来”等20余幅...,河北广宗：巧手剪纸致敬抗疫英雄,新华社


In [9]:
t = df[df['video_id'] == 'v0201fd10000bpf0vrdjfrmk75634a30']

index = 0
t.iloc[index]['title'], t.iloc[index]['abstract'], t.iloc[index]['video_id']

('战“疫”日记｜你的样子',
 '漳州市第二批支援湖北医疗队队员、漳州正兴医院主管护师张书贤记录到：我收到了一份特别的礼物。那是一段患者发来的视频，题目叫《你的样子》，视频里的我们穿着防护服，或在量血压、查体征、写护记、分发三餐，看着看着，心里暖洋洋的，那是一种被人记在心尖的感动和幸福。',
 'v0201fd10000bpf0vrdjfrmk75634a30')

# 上传到Youtube

In [ ]:
import http
import httplib2
import os
import random
import sys
import time

import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors

from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaFileUpload
from oauth2client.client import flow_from_clientsecrets
from oauth2client.file import Storage
from oauth2client.tools import argparser, run_flow


# Explicitly tell the underlying HTTP transport library not to retry, since
# we are handling retry logic ourselves.
httplib2.RETRIES = 1

# Maximum number of times to retry before giving up.
MAX_RETRIES = 10

# Always retry when these exceptions are raised.
RETRIABLE_EXCEPTIONS = (httplib2.HttpLib2Error, IOError, http.client.NotConnected,
  http.client.IncompleteRead, http.client.ImproperConnectionState,
  http.client.CannotSendRequest, http.client.CannotSendHeader,
  http.client.ResponseNotReady, http.client.BadStatusLine)

# Always retry when an apiclient.errors.HttpError with one of these status
# codes is raised.
RETRIABLE_STATUS_CODES = [500, 502, 503, 504]

# The CLIENT_SECRETS_FILE variable specifies the name of a file that contains
# the OAuth 2.0 information for this application, including its client_id and
# client_secret. You can acquire an OAuth 2.0 client ID and client secret from
# the Google API Console at
# https://console.developers.google.com/.
# Please ensure that you have enabled the YouTube Data API for your project.
# For more information about using OAuth2 to access the YouTube Data API, see:
#   https://developers.google.com/youtube/v3/guides/authentication
# For more information about the client_secrets.json file format, see:
#   https://developers.google.com/api-client-library/python/guide/aaa_client_secrets
CLIENT_SECRETS_FILE = "client_secrets.json"

# This OAuth 2.0 access scope allows an application to upload files to the
# authenticated user's YouTube channel, but doesn't allow other types of access.
YOUTUBE_UPLOAD_SCOPE = "https://www.googleapis.com/auth/youtube.upload"
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"

# This variable defines a message to display if the CLIENT_SECRETS_FILE is
# missing.
MISSING_CLIENT_SECRETS_MESSAGE = "Missing client secrets message"

VALID_PRIVACY_STATUSES = ("public", "private", "unlisted")


def get_authenticated_service():
  # flow = flow_from_clientsecrets(CLIENT_SECRETS_FILE,
  #   scope=YOUTUBE_UPLOAD_SCOPE,
  #   message=MISSING_CLIENT_SECRETS_MESSAGE)

  # storage = Storage("%s-oauth2.json" % sys.argv[0])
  # credentials = storage.get()

  # if credentials is None or credentials.invalid:
  #   credentials = run_flow(flow, storage, args)

  flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
        CLIENT_SECRETS_FILE, YOUTUBE_UPLOAD_SCOPE)
  credentials = flow.run_console()

  # return build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,
  #   http=credentials.authorize(httplib2.Http()))
  return googleapiclient.discovery.build(
        YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, credentials=credentials)

def initialize_upload(youtube, options):
  tags = None
  if options.keywords:
    tags = options.keywords.split(",")

  body=dict(
    snippet=dict(
      title=options.title,
      description=options.description,
      tags=tags,
      categoryId=options.category
    ),
    status=dict(
      privacyStatus=options.privacyStatus
    )
  )

  # Call the API's videos.insert method to create and upload the video.
  insert_request = youtube.videos().insert(
    part=",".join(body.keys()),
    body=body,
    # The chunksize parameter specifies the size of each chunk of data, in
    # bytes, that will be uploaded at a time. Set a higher value for
    # reliable connections as fewer chunks lead to faster uploads. Set a lower
    # value for better recovery on less reliable connections.
    #
    # Setting "chunksize" equal to -1 in the code below means that the entire
    # file will be uploaded in a single HTTP request. (If the upload fails,
    # it will still be retried where it left off.) This is usually a best
    # practice, but if you're using Python older than 2.6 or if you're
    # running on App Engine, you should set the chunksize to something like
    # 1024 * 1024 (1 megabyte).
    media_body=MediaFileUpload(options.file, chunksize=-1, resumable=True)
  )

  resumable_upload(insert_request)

# This method implements an exponential backoff strategy to resume a
# failed upload.
def resumable_upload(insert_request):
  response = None
  error = None
  retry = 0
  while response is None:
    try:
      print("Uploading file...")
      status, response = insert_request.next_chunk()
      if response is not None:
        if 'id' in response:
          print("Video id '%s' was successfully uploaded." % response['id'])
        else:
          exit("The upload failed with an unexpected response: %s" % response)
    except HttpError as e:
      if e.resp.status in RETRIABLE_STATUS_CODES:
        error = "A retriable HTTP error %d occurred:\n%s" % (e.resp.status,
                                                             e.content)
      else:
        raise
    except RETRIABLE_EXCEPTIONS as e:
      error = "A retriable error occurred: %s" % e

    if error is not None:
      print(error)
      retry += 1
      if retry > MAX_RETRIES:
        exit("No longer attempting to retry.")

      max_sleep = 2 ** retry
      sleep_seconds = random.random() * max_sleep
      print("Sleeping %f seconds and then retrying..." % (sleep_seconds))
      time.sleep(sleep_seconds)

In [ ]:
argparser.add_argument("--file", required=True, help="Video file to upload")
argparser.add_argument("--title", help="Video title", default="Test Title")
argparser.add_argument("--description", help="Video description", default="Test Description")
argparser.add_argument("--category", default="22", help="Numeric video category. " + "See https://developers.google.com/youtube/v3/docs/videoCategories/list")
argparser.add_argument("--keywords", help="Video keywords, comma separated", default="")
argparser.add_argument("--privacyStatus", choices=VALID_PRIVACY_STATUSES, default=VALID_PRIVACY_STATUSES[0], help="Video privacy status.")

In [ ]:
youtube = get_authenticated_service()

In [ ]:
data

In [ ]:
for record in data:
    params = [
        '--file=/content/%s.mp4' % (record['video_id']),
        '--title=%s' % (record['title']),
        '--description=%s' % (record['abstract']),
        '--category=25',
        '--keywords=News'
    ]
    print(params)

    if params[0] in ['--file=/content/v02004930000bpdsqb20ifkna9sj0oq0.mp4', '--file=/content/v020047e0000bpdsamvijqt1h4rr196g.mp4']:
        continue

    args = argparser.parse_args(args=params)
    
    try:
        initialize_upload(youtube, args)
    except HttpError as e:
        print(e)
        break
    #     print("An HTTP error %d occurred:\n%s" % (e.resp.status, e.content))

    # time.sleep(10)

In [ ]:
!ls -all

# youtube 类目id

In [ ]:
# -*- coding: utf-8 -*-

# Sample Python code for youtube.videoCategories.list
# See instructions for running these code samples locally:
# https://developers.google.com/explorer-help/guides/code_samples#python

import os

import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors

scopes = ["https://www.googleapis.com/auth/youtube.readonly"]

def category():
    # Disable OAuthlib's HTTPS verification when running locally.
    # *DO NOT* leave this option enabled in production.
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

    api_service_name = "youtube"
    api_version = "v3"
    client_secrets_file = "client_secrets.json"

    # Get credentials and create an API client
    flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
        client_secrets_file, scopes)
    credentials = flow.run_console()
    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, credentials=credentials)

    request = youtube.videoCategories().list(
        part="snippet",
        regionCode="US"
    )
    response = request.execute()

    print(response)

category()